In [222]:
import numpy as np
import pandas as pd
import pathlib as p
import seaborn as sb
import matplotlib.pyplot as plt
import pathlib
from pathlib import Path
from datetime import datetime
from IPython.display import Markdown as md
from IPython.display import Math as m


In [223]:
%matplotlib inline

In [224]:
def extractParentDir(dataFileName):
    purePath = pathlib.PurePath(dataFileName)
    parentDir2 = purePath.parent.parent
    return parentDir2

In [225]:
todayVal = datetime.today()
timeStampStr = todayVal.strftime("%y-%m-%d_%H-%M-%S.%f")
data_dir = r"C:\work\dev\dECMT_src\data_all\COVID19_Data\Current"
parentDir2 = extractParentDir(data_dir)
outputPath = Path(parentDir2) / "Output_Covid19_Analysis"/f"{timeStampStr}_bipartite"

In [226]:
comorbidFN = p.Path(data_dir) / "REACT_Comorbidities_unpivoted.csv"
demogFN = p.Path(data_dir) / "REACT_Demographics.csv"
comorbidDF:pd.DataFrame = pd.read_csv(comorbidFN)

In [227]:
comorbidDF:pd.DataFrame = pd.read_csv(comorbidFN)
demogDF:pd.DataFrame = pd.read_csv(demogFN)
covidDemogDF = pd.read_csv(p.Path(data_dir)/ 'REACT_COVID_Demographics.csv')
# **************************
lab_results_df = pd.read_csv(p.Path(data_dir) / 'REACT_LabResults.csv')
# pharmacy_data_df = pd.read_csv(p.Path(data_dir) / 'REACT_PharmacyData.csv')
covid_test_df = pd.read_csv(p.Path(data_dir) / 'REACT_UHSCOVIDTest_processed.csv')


In [228]:
demogDF.describe()

,PATIENT_AGE,GENDER,IS_PREGNANT,HEIG,WEIG,BMI,LOS,LOS_PREPOS
count,614.000000,614.000000,614.000000,277.000000,593.000000,271.000000,614.000000,614.000000
mean,67.929967,1.571661,0.009772,167.709531,75.407437,26.277343,15.838322,3.847264
std,19.635836,0.495241,0.098449,14.785532,21.033713,5.907941,17.310716,9.651141
min,0.000000,1.000000,0.000000,66.000000,0.000000,12.860000,0.000000,-0.370000
25%,55.250000,1.000000,0.000000,160.000000,62.700000,22.255000,4.622500,-0.080000
50%,72.000000,2.000000,0.000000,168.000000,75.000000,25.510000,10.095000,0.010000
75%,83.000000,2.000000,0.000000,177.800000,86.000000,29.895000,21.827500,3.705000
max,103.000000,2.000000,1.000000,198.000000,192.000000,50.560000,173.580000,127.140000


In [229]:
demogDF.columns

Index(['STUDY_ID', 'PATIENT_AGE', 'DOB', 'DATE_OF_DEATH', 'DOD_DATE', 'GENDER',
       'ETHNIC_GROUP', 'SMOKING_HISTORY', 'POSTCODE', 'IS_PREGNANT', 'HEIG',
       'WEIG', 'BMI', 'FIRST_POS_DATE', 'FIRST_POS_DATE_R', 'FIRST_POS_TIME_R',
       'ADMIT_DATETIME', 'ADM_DATE_R', 'ADM_TIME_R', 'DISCHARGEDATE',
       'DISCHARGE_DATE_R', 'DISCHARGE_TIME_R', 'LOS', 'LOS_PREPOS', 'READM28',
       'READM_DATETIME', 'READM_DATE', 'READM_TIME'],
      dtype='object')

In [230]:
demogDF.head()

,STUDY_ID,PATIENT_AGE,DOB,DATE_OF_DEATH,DOD_DATE,GENDER,ETHNIC_GROUP,SMOKING_HISTORY,POSTCODE,IS_PREGNANT,...,ADM_TIME_R,DISCHARGEDATE,DISCHARGE_DATE_R,DISCHARGE_TIME_R,LOS,LOS_PREPOS,READM28,READM_DATETIME,READM_DATE,READM_TIME
0,UHSCOVID_00989a42,64.0,01/1956,NaN,NaN,2,A,NR|NR,SO19,0,...,17:19:00,2020-04-02 10:21:00,02/04/2020,10:21:00,6.71,-0.10,N,NaN,NaN,NaN
1,UHSCOVID_00ac1e32,73.0,10/1946,NaN,NaN,2,A,Ex-smoker|NR,SO21,0,...,18:00:00,2020-05-19 12:56:00,19/05/2020,12:56:00,41.79,4.00,N,NaN,NaN,NaN
2,UHSCOVID_00cb7bdd,76.0,03/1944,2020-04-13 00:00:00,13/04/2020,1,A,NR|NR,SO16,0,...,17:56:00,2020-04-13 02:15:00,13/04/2020,02:15:00,2.35,-0.04,N,NaN,NaN,NaN
3,UHSCOVID_01add35c,67.0,10/1952,2020-05-27 00:00:00,27/05/2020,1,A,NR|NR,SO19,0,...,10:17:00,2020-05-23 18:15:00,23/05/2020,18:15:00,1.33,0.21,Y,2020-05-26 09:41:00,26/05/2020,09:41:00
4,UHSCOVID_01f7838d,97.0,01/1923,NaN,NaN,2,A,NR|NR,SO16,0,...,09:00:00,2020-05-17 16:00:00,17/05/2020,16:00:00,13.29,7.13,N,NaN,NaN,NaN


In [231]:
demogDF['Death'] = demogDF['DATE_OF_DEATH'].notna()

In [232]:
demogDF2=demogDF.groupby('STUDY_ID').agg(Death=('Death','any'))
demogDF2

,Death
STUDY_ID,
UHSCOVID_00989a42,False
UHSCOVID_00ac1e32,False
UHSCOVID_00cb7bdd,True
UHSCOVID_01add35c,True
UHSCOVID_01f7838d,False
...,...
UHSCOVID_fc872d82,False
UHSCOVID_fd016c0e,False
UHSCOVID_fd4da220,False


# Renal disease flag


In [233]:
comorbidDF.describe()



,STATUS
count,1628.0
mean,1.0
std,0.0
min,1.0
25%,1.0
50%,1.0
75%,1.0
max,1.0


In [234]:
comorbidDF.head()

,STUDY_ID,COMORBIDITY,STATUS
0,UHSCOVID_00989a42,COPD,1
1,UHSCOVID_0520de1c,COPD,1
2,UHSCOVID_0744a86c,COPD,1
3,UHSCOVID_0874d194,COPD,1
4,UHSCOVID_0b401459,COPD,1


In [235]:
comorbidDF.columns



Index(['STUDY_ID', 'COMORBIDITY', 'STATUS'], dtype='object')

In [236]:
comorbidDF['RENAL_D_PRESENT'] = (comorbidDF['COMORBIDITY'] == 'RENAL_D')


In [237]:
comorbidDF2 = comorbidDF.groupby('STUDY_ID').agg(RENAL_D_PRESENT=('RENAL_D_PRESENT',any))
comorbidDF2

,RENAL_D_PRESENT
STUDY_ID,
UHSCOVID_00989a42,False
UHSCOVID_00ac1e32,False
UHSCOVID_00cb7bdd,False
UHSCOVID_01add35c,False
UHSCOVID_01f7838d,True
...,...
UHSCOVID_fb3b62bb,True
UHSCOVID_fc872d82,False
UHSCOVID_fd4da220,False


In [238]:
# u.convertDatesWTableName(covidDemogDF,'demographics_df')

# Association between Renal disease and Death

In [239]:
demogDF2.head()

,Death
STUDY_ID,
UHSCOVID_00989a42,False
UHSCOVID_00ac1e32,False
UHSCOVID_00cb7bdd,True
UHSCOVID_01add35c,True
UHSCOVID_01f7838d,False


In [240]:
comorbidDF2.head()

,RENAL_D_PRESENT
STUDY_ID,
UHSCOVID_00989a42,False
UHSCOVID_00ac1e32,False
UHSCOVID_00cb7bdd,False
UHSCOVID_01add35c,False
UHSCOVID_01f7838d,True


In [241]:
dfRes:pd.DataFrame = pd.merge(demogDF2, comorbidDF2, on='STUDY_ID', how='inner')

In [242]:
dfRes = dfRes.reset_index()


In [243]:
dfRes.columns


Index(['STUDY_ID', 'Death', 'RENAL_D_PRESENT'], dtype='object')

In [244]:
dfRes.head()

,STUDY_ID,Death,RENAL_D_PRESENT
0,UHSCOVID_00989a42,False,False
1,UHSCOVID_00ac1e32,False,False
2,UHSCOVID_00cb7bdd,True,False
3,UHSCOVID_01add35c,True,False
4,UHSCOVID_01f7838d,False,True


In [245]:
md(f"Counts of stratified cases")

Counts of stratified cases

In [246]:
# crosstabRenalDeath = pd.crosstab(dfRes3['Death'],dfRes3['RENAL_D_PRESENT'],margins=True,margins_name='Total')
crosstabRenalDeath = pd.crosstab(dfRes['Death'],dfRes['RENAL_D_PRESENT'])
crosstabRenalDeath


RENAL_D_PRESENT,False,True
Death,,
False,252,91
True,96,75


In [247]:
crosstabRenalDeath2 = pd.crosstab(dfRes['Death'],dfRes['RENAL_D_PRESENT'],margins=True)
crosstabRenalDeath2



RENAL_D_PRESENT,False,True,All
Death,,,
False,252,91,343
True,96,75,171
All,348,166,514


In [248]:
crosstabRenalDeathNorm = pd.crosstab(dfRes['Death'],dfRes['RENAL_D_PRESENT'],normalize=True,margins=True)
crosstabRenalDeathNorm

RENAL_D_PRESENT,False,True,All
Death,,,
False,0.490272,0.177043,0.667315
True,0.186770,0.145914,0.332685
All,0.677043,0.322957,1.000000


In [249]:
crosstabRenalDeath.index

Index([False, True], dtype='object', name='Death')

In [250]:
crosstabRenalDeath.columns

Index([False, True], dtype='object', name='RENAL_D_PRESENT')

In [251]:
crosstabRenalDeath.reset_index()
crosstabRenalDeath

RENAL_D_PRESENT,False,True
Death,,
False,252,91
True,96,75


In [252]:
# crosstabRenalDeath.loc['RENAL_D_PRESENT']

## Chi2 test on significance of renal disease for death outcome.


In [253]:
from scipy.stats import chi2_contingency

# defining the table
# data = [crosstabRenalDeath.loc['False',:].values().reshape(1,-1), [234, 242, 232]]
stat, p, dof, expected = chi2_contingency(crosstabRenalDeath)
print(f"stat = {stat}")
print(f"p = {p}")
print(f"dof = {dof}")
print(f"expected = {expected}")
md(f"Since p={p:.1g}<0.05, based on Chi2 test we can reject hypothesis that death as outcome does not depend on the presence of a renal disease.")

stat = 14.88914922319227
p = 0.00011401693102745145
dof = 1
expected = [[232.22568093 110.77431907]
 [115.77431907  55.22568093]]


Since p=0.0001<0.05, based on Chi2 test we can reject hypothesis that death as outcome does not depend on the presence of a renal disease.

## Power of evidence

In [254]:
crosstabRenalDeathNorm

RENAL_D_PRESENT,False,True,All
Death,,,
False,0.490272,0.177043,0.667315
True,0.186770,0.145914,0.332685
All,0.677043,0.322957,1.000000


In [255]:
md(f" Conditional probabilities")

 Conditional probabilities

In [256]:
crosstabRenalDeath


RENAL_D_PRESENT,False,True
Death,,
False,252,91
True,96,75


In [257]:
idx=pd.IndexSlice
R_T = crosstabRenalDeath.loc[:,idx[True]].sum()

In [258]:
crosstabRenalDeathNorm

RENAL_D_PRESENT,False,True,All
Death,,,
False,0.490272,0.177043,0.667315
True,0.186770,0.145914,0.332685
All,0.677043,0.322957,1.000000


In [259]:
R_T
P_R_T = crosstabRenalDeathNorm.loc['All',True]
P_D_T = crosstabRenalDeathNorm.loc[True,'All']
P_R_F = crosstabRenalDeathNorm.loc['All',False]
P_D_F = crosstabRenalDeathNorm.loc[False,'All']
PDTGRT = crosstabRenalDeathNorm.loc[True,True]/P_R_T
PDFGRT = 1-PDTGRT
# ********************************
PDFGRF = crosstabRenalDeathNorm.loc[False,False]/P_R_F
PDTGRF = 1-PDFGRF
# **********
PRTGDT =  crosstabRenalDeathNorm.loc[True,True]/P_D_T
PRFGDT = 1-PRTGDT
PRFGDF = crosstabRenalDeathNorm.loc[False,False]/P_D_F
PRTGDF = 1-PRFGDF
WoERTGD = PRTGDT/PRTGDF
WoERFGD = PRFGDT/PRFGDF
# ********************************
print(f"PRT={P_R_T} PRF = {P_R_F} ")
print(f"PDTGRT={PDTGRT} PDFGRT={PDFGRT}")
print(f"WoERFGD={WoERFGD}")
# ********************************
print(f"PRT={P_R_T} PRF = {P_R_F} ")
print(f"PDTGRF={PDTGRF} PDFGRT={PDFGRF}")
print(f"WoERTGD={WoERTGD}")

PRT=0.3229571984435798 PRF = 0.6770428015564203 
PDTGRT=0.45180722891566266 PDFGRT=0.5481927710843373
WoERFGD=0.7641325536062378
PRT=0.3229571984435798 PRF = 0.6770428015564203 
PDTGRF=0.27586206896551735 PDFGRT=0.7241379310344827
WoERTGD=1.6531713900134957


# Value of information (WoE)
The more is WoE different from 0 the more informative is renal disease for prediction of death, while positive value means support for selected result and negative value means the support of other result by evidence.



$$WoE(Evidence|Target) =100*log_{10}\Bigg  (\frac{P(Evidence|Target)}{P(Evidence|\neg Target)}\Bigg)$$

$$WoE(\neg Evidence|Target) =100*log_{10}\Bigg(\frac{P(\neg Evidence|Target)}{P(\neg Evidence|\neg Target)}\Bigg)$$

$$WoE \in (-\infty,\infty )$$$


In [260]:
md(f"Weight of Evidence, WoE(Renal_disease=Absent|Death) = {100*np.log10(WoERFGD)}")

Weight of Evidence, WoE(Renal_disease=Absent|Death) = -11.683129809135904

In [261]:
md(f"Weight of Evidence, WoE(Renal_disease=Present|Death) = {100*np.log10(WoERTGD)}")

Weight of Evidence, WoE(Renal_disease=Present|Death) = 21.831788072122322

In [262]:
RENAL_D_PRESENT = False
crosstabRenalDeath.loc[:,idx[RENAL_D_PRESENT]]

Death
False    252
True      96
dtype: int64

In [263]:
Died=False
crosstabRenalDeath.loc[Died,idx[RENAL_D_PRESENT]]


252

In [264]:
crosstabRenalDeath.loc[False,idx[False]]


252